###URL Context

Extract the codeListValue or other possible identifying information for CI_OnlineResource elements - where and what. Just to note what kind of text is used, from which servers, in which kinds of sections.

In [ ]:
%reload_ext autoreload
%autoreload 2

import json as js  # name conflict with sqla
import sqlalchemy as sqla
from sqlalchemy.orm import sessionmaker

from mpp.models import Response

from lxml import etree

In [ ]:
# grab the clean text from the rds
with open('../local/big_rds.conf', 'r') as f:
    conf = js.loads(f.read())

# our connection
engine = sqla.create_engine(conf.get('connection'))
Session = sessionmaker()
Session.configure(bind=engine)
session = Session()

In [ ]:
# get any iso
sql_pttn = """
with i
as (
    select d.response_id, trim(both '"' from (e.value->'protocol')::text) as protocol
    from identities d, jsonb_array_elements(d.identity::jsonb) e
    where d.identity is not null 
        and e.value->>'protocol' = 'ISO'
)

select r.id, r.source_url, r.cleaned_content
from responses r join i on i.response_id = r.id
limit %s
offset %s;
"""

In [ ]:
xp = "//*[local-name()='CI_OnlineResource']"

START = 0
STOP = 20000
LIMIT = 100

for i in xrange(START, STOP, LIMIT):
    sql = sql_pttn % (LIMIT, i)
    
    for response_id, source_url, cleaned_content in session.execute(sql):
        try:
            xml = etree.fromstring(cleaned_content.encode('utf-8'))
        except:
            print 'failed to parse', response_id, source_url
            continue
        
        